In [5]:
import pandas as pd

file  = './KBO_2019_player_gamestats.xlsx'
raw = pd.read_excel(file)
raw.head()

,Team,Name,DOB,Date,Opponent,Result,GS,AB,R,H,...,SH,SF,AVG,OBP,SLG,OPS,DIPS,avLI,RE24,WPA
0,Doosan,페르난데스,1988-04-27,2021-03-23,Hanhwa,W 5:4,1,4,0,2,...,0,0,0.500,0.500,0.750,1.250,19,1.98,1.65,0.429
1,Doosan,페르난데스,1988-04-27,2021-03-24,Hanhwa,L 1:11,1,2,0,0,...,0,0,0.333,0.333,0.500,0.833,4,0.77,-0.36,-0.038
2,Doosan,페르난데스,1988-04-27,2021-03-26,kiwoom,W 7:2,1,2,2,0,...,0,0,0.250,0.400,0.375,0.775,16,1.56,0.98,0.146
3,Doosan,페르난데스,1988-04-27,2021-03-27,kiwoom,W 3:2,1,4,1,3,...,0,0,0.417,0.500,0.500,1.000,11,1.53,1.29,0.189
4,Doosan,페르난데스,1988-04-27,2021-03-28,kiwoom,L 4:5,1,3,1,0,...,0,0,0.333,0.474,0.400,0.874,23,2.04,-0.30,-0.166


# Q) Who's performance is steady regardless of weather (esp. hot summer)
- monthly record

In [15]:
#extract only month from 'Date' and create separate 'month' column 
month_list = []
for monthdate in raw['Date']:
    month = monthdate.month
    month_list.append(month)
raw['Month'] = month_list

In [18]:
#select columns for analysis
columns_select = ['Team', 'Name', 'DOB','Date', 'Opponent','AB','H','HR', 'TB', 'RBI','BB', 'HBP', 'SF', 'Month']

data = raw[columns_select]
data.head()

,Team,Name,DOB,Date,Opponent,AB,H,HR,TB,RBI,BB,HBP,SF,Month
0,Doosan,페르난데스,1988-04-27,2021-03-23,Hanhwa,4,2,0,3,3,0,0,0,3
1,Doosan,페르난데스,1988-04-27,2021-03-24,Hanhwa,2,0,0,0,0,0,0,0,3
2,Doosan,페르난데스,1988-04-27,2021-03-26,kiwoom,2,0,0,0,1,2,0,0,3
3,Doosan,페르난데스,1988-04-27,2021-03-27,kiwoom,4,3,0,3,1,0,0,0,3
4,Doosan,페르난데스,1988-04-27,2021-03-28,kiwoom,3,0,0,0,0,2,0,0,3


In [19]:
#Using pivot table, view monthly performance  
data_player_month = data.pivot_table(index = ['Team','Name','DOB', 'Month'], 
                               values = ['AB','H','HR','TB','RBI','BB','HBP','SF'], 
                              aggfunc = 'sum', fill_value = 0
                                )
data_player_month

AB  BB   H  HBP  HR  RBI  SF  TB
Team   Name DOB        Month                                  
Doosan 국해성  1989-10-08 3       2   1   0    1   0    0   0   0
                       4       1   0   0    0   0    0   0   0
                       5      21   2   4    0   0    2   0   6
                       6       3   1   0    0   0    0   0   0
                       8       6   1   0    0   0    0   0   0
...                           ..  ..  ..  ...  ..  ...  ..  ..
lotte  허일   1992-11-20 5      71   8  18    1   0    8   0  19
                       6      24   2   6    0   0    1   0   6
                       8      10   1   2    0   0    0   0   3
                       9      18   0   4    0   0    2   0   4
       황진수  1989-02-15 5       3   1   0    0   0    0   0   0

[1261 rows x 8 columns]

In [20]:
data_player_month = data_player_month.reset_index()
data_player_month

,Team,Name,DOB,Month,AB,BB,H,HBP,HR,RBI,SF,TB
0,Doosan,국해성,1989-10-08,3,2,1,0,1,0,0,0,0
1,Doosan,국해성,1989-10-08,4,1,0,0,0,0,0,0,0
2,Doosan,국해성,1989-10-08,5,21,2,4,0,0,2,0,6
3,Doosan,국해성,1989-10-08,6,3,1,0,0,0,0,0,0
4,Doosan,국해성,1989-10-08,8,6,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1256,lotte,허일,1992-11-20,5,71,8,18,1,0,8,0,19
1257,lotte,허일,1992-11-20,6,24,2,6,0,0,1,0,6
1258,lotte,허일,1992-11-20,8,10,1,2,0,0,0,0,3
1259,lotte,허일,1992-11-20,9,18,0,4,0,0,2,0,4


In [21]:
#func to calculate AVG/OBP/SLG/OPS of a batter 
def cal_hit(df):
    '''
    - 타율 : 공을 쳐서 나가는 비율 --> 안타 / 타수
    - 출루율: 진루해서 나가는 비율 -->  (안타+볼넷+몸에맞는볼)/(타수+볼넷+몸에맞는볼+희생플라이)
    - 장타율 : 타율에 진루한 베이스 가중치 추가 -->   루타 / 타수
    '''
    
    df['AVG'] = df['H'] / df['AB']
    df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / (df['AB'] + df['HBP'] + df['SF'])
    df['SLG'] = df['TB'] / df['AB']
    df['OPS'] = df['OBP'] + df['SLG']
    return df

In [22]:
# data_player_month  evaluate cal_hit of each batter
player_month_stat = cal_hit(data_player_month)
player_month_stat = player_month_stat.dropna()
player_month_stat

,Team,Name,DOB,Month,AB,BB,H,HBP,HR,RBI,SF,TB,AVG,OBP,SLG,OPS
0,Doosan,국해성,1989-10-08,3,2,1,0,1,0,0,0,0,0.000000,0.666667,0.000000,0.666667
1,Doosan,국해성,1989-10-08,4,1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
2,Doosan,국해성,1989-10-08,5,21,2,4,0,0,2,0,6,0.190476,0.285714,0.285714,0.571429
3,Doosan,국해성,1989-10-08,6,3,1,0,0,0,0,0,0,0.000000,0.333333,0.000000,0.333333
4,Doosan,국해성,1989-10-08,8,6,1,0,0,0,0,0,0,0.000000,0.166667,0.000000,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,lotte,허일,1992-11-20,5,71,8,18,1,0,8,0,19,0.253521,0.375000,0.267606,0.642606
1257,lotte,허일,1992-11-20,6,24,2,6,0,0,1,0,6,0.250000,0.333333,0.250000,0.583333
1258,lotte,허일,1992-11-20,8,10,1,2,0,0,0,0,3,0.200000,0.300000,0.300000,0.600000
1259,lotte,허일,1992-11-20,9,18,0,4,0,0,2,0,4,0.222222,0.222222,0.222222,0.444444


In [24]:
#montly OBP
month_pivot = player_month_stat.pivot_table(index = ['Team','Name','DOB'],
                             columns = 'Month',
                             values = 'OBP')
month_pivot = month_pivot.reset_index()
month_pivot

Month,Team,Name,DOB,3,4,5,6,7,8,9,10
0,Doosan,국해성,1989-10-08,0.666667,0.000000,0.285714,0.333333,NaN,0.166667,1.000000,0.500000
1,Doosan,김경호,1995-07-31,NaN,NaN,0.153846,0.333333,NaN,0.142857,NaN,NaN
2,Doosan,김대한,2000-12-06,0.333333,0.200000,0.142857,NaN,NaN,NaN,NaN,NaN
3,Doosan,김인태,1994-07-03,NaN,0.000000,0.333333,NaN,NaN,0.250000,0.384615,0.500000
4,Doosan,김재호,1985-03-21,0.269231,0.441176,0.537313,0.515152,0.415094,0.442857,0.307692,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
224,lotte,정준혁,1994-04-01,NaN,NaN,NaN,0.333333,NaN,NaN,0.142857,1.000000
225,lotte,정훈,1987-07-18,0.000000,0.428571,0.375000,0.384615,0.333333,0.333333,0.326531,0.250000
226,lotte,채태인,1982-10-11,0.409091,0.230769,0.279070,NaN,0.571429,0.350000,NaN,NaN
227,lotte,허일,1992-11-20,NaN,0.346154,0.375000,0.333333,NaN,0.300000,0.222222,NaN
